# P3: Deffault Payment

> José María Reyes Figueroa

In [4]:
import numpy as np 
import pandas as pd 
import pickle

#Visualization

import cufflinks as cf
cf.go_offline()

#Proprocessing

from sklearn.pipeline import Pipeline as skpipe
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from varclushi import VarClusHi
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline

#Models

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

#Metrics

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc

pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('./train_p3.csv')

In [3]:
data

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,18125,220000,2,3,2,32,0,0,0,0,0,0,209259,192365,184198,171391,178742,164793,7000,15000,6000,10000,10000,7000,0
1,3983,220000,2,2,1,41,2,0,0,2,0,0,27094,27819,30363,29579,48933,97187,1465,3009,0,20000,50000,1240,0
2,19251,80000,2,1,2,27,-1,-1,-1,0,-1,-1,3199,1205,917,917,702,3099,1325,917,0,702,3099,0,0
3,4024,20000,2,2,2,38,1,2,2,4,3,2,10683,12729,14734,14190,13721,13848,2500,2501,0,0,441,1,0
4,20610,100000,2,1,2,28,0,0,0,0,0,0,102697,100462,101879,100999,101188,99328,4600,4000,4000,4000,4000,4400,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,13019,110000,1,1,2,28,0,0,0,0,0,0,102282,104196,105922,108820,66883,68083,5200,5000,5000,2100,2000,2000,0
20996,27065,420000,2,2,2,25,-1,0,0,0,0,0,28206,122212,117835,112705,111357,108306,100000,4013,3822,4086,3800,3600,0
20997,5873,50000,1,2,1,34,0,0,0,0,0,0,48976,47404,46338,14234,19545,20173,2023,3105,3119,11000,909,3000,0
20998,21449,100000,2,1,2,27,-2,-2,-2,-2,-2,-2,-2000,5555,0,0,0,0,7555,0,0,0,0,0,0


In [4]:
data.columns = ['id', 'limit_bal', 'sex', 'education', 'marriage', 'age', 'pay1', 'pay2', 'pay3', 'pay4', 'pay5', 'pay6', 'bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6', 'pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6', 'default payment next month']

In [5]:
data

,id,limit_bal,sex,education,marriage,age,pay1,pay2,pay3,pay4,pay5,pay6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default payment next month
0,18125,220000,2,3,2,32,0,0,0,0,0,0,209259,192365,184198,171391,178742,164793,7000,15000,6000,10000,10000,7000,0
1,3983,220000,2,2,1,41,2,0,0,2,0,0,27094,27819,30363,29579,48933,97187,1465,3009,0,20000,50000,1240,0
2,19251,80000,2,1,2,27,-1,-1,-1,0,-1,-1,3199,1205,917,917,702,3099,1325,917,0,702,3099,0,0
3,4024,20000,2,2,2,38,1,2,2,4,3,2,10683,12729,14734,14190,13721,13848,2500,2501,0,0,441,1,0
4,20610,100000,2,1,2,28,0,0,0,0,0,0,102697,100462,101879,100999,101188,99328,4600,4000,4000,4000,4000,4400,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,13019,110000,1,1,2,28,0,0,0,0,0,0,102282,104196,105922,108820,66883,68083,5200,5000,5000,2100,2000,2000,0
20996,27065,420000,2,2,2,25,-1,0,0,0,0,0,28206,122212,117835,112705,111357,108306,100000,4013,3822,4086,3800,3600,0
20997,5873,50000,1,2,1,34,0,0,0,0,0,0,48976,47404,46338,14234,19545,20173,2023,3105,3119,11000,909,3000,0
20998,21449,100000,2,1,2,27,-2,-2,-2,-2,-2,-2,-2000,5555,0,0,0,0,7555,0,0,0,0,0,0


In [6]:
data['sex'].value_counts()

sex
2    12709
1     8291
Name: count, dtype: int64

In [7]:
data['education'].value_counts()

education
2    9785
1    7429
3    3468
5     189
4      82
6      37
0      10
Name: count, dtype: int64

In [8]:
data['education'] = data['education'].replace({0:5, 6:5})

In [9]:
data['education'].value_counts()

education
2    9785
1    7429
3    3468
5     236
4      82
Name: count, dtype: int64

In [10]:
data['marriage'].value_counts()

marriage
2    11188
1     9551
3      224
0       37
Name: count, dtype: int64

In [11]:
data['marriage'] = data['marriage'].replace({0:3})

In [12]:
data['marriage'].value_counts()

marriage
2    11188
1     9551
3      261
Name: count, dtype: int64

In [13]:
data['age'].describe()

count    21000.000000
mean        35.502095
std          9.211036
min         21.000000
25%         28.000000
50%         34.000000
75%         41.000000
max         79.000000
Name: age, dtype: float64

In [14]:
data['default payment next month'].value_counts()

default payment next month
0    16445
1     4555
Name: count, dtype: int64

In [15]:
data['pay1'].value_counts()

pay1
 0    10313
-1     3943
 1     2600
-2     1957
 2     1873
 3      212
 4       54
 5       21
 8       13
 6        7
 7        7
Name: count, dtype: int64

In [16]:
for i in range(1, 7):
    data['pay'+str(i)] = data['pay'+str(i)].replace({-2:-1, 0:-1})

In [17]:
data['pay1'].value_counts()

pay1
-1    16213
 1     2600
 2     1873
 3      212
 4       54
 5       21
 8       13
 6        7
 7        7
Name: count, dtype: int64

In [18]:
defs = data[data['default payment next month'] == 1]['id']

In [19]:
data[data['id'].isin(defs)]

,id,limit_bal,sex,education,marriage,age,pay1,pay2,pay3,pay4,pay5,pay6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default payment next month
11,26689,420000,2,2,1,41,-1,-1,-1,-1,-1,-1,373978,389754,366938,361024,354813,344221,33808,14996,13209,12890,11081,16298,1
12,28758,50000,1,1,1,49,1,2,-1,-1,-1,-1,27231,26500,27533,28569,28901,31095,0,1764,3400,1100,2660,0,1
16,3993,50000,2,1,2,29,3,3,2,3,2,2,49624,48530,50255,49104,51044,50933,0,2800,0,3000,800,3000,1
23,19975,30000,2,2,2,46,-1,-1,2,2,2,-1,19495,22254,21616,26695,25970,26402,3400,0,5800,0,1000,1400,1
24,19688,180000,2,2,1,39,2,2,2,2,2,-1,177113,180628,176137,183007,179505,105229,7800,0,11700,0,3775,7000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20967,11421,500000,2,1,2,28,2,-1,-1,2,-1,-1,98541,102052,111690,78070,78376,80912,6000,13151,0,3000,5000,10000,1
20970,18829,70000,2,1,2,23,-1,-1,-1,-1,-1,-1,7668,8878,10077,11203,11320,11487,1500,1500,1600,600,500,600,1
20974,27256,260000,2,1,2,28,-1,-1,-1,-1,-1,-1,187068,193245,204730,246985,240543,227725,10360,60000,100008,10000,11700,5000,1
20985,14362,140000,1,1,2,34,2,2,2,2,2,2,124640,131032,132065,129307,131154,134470,9900,4700,800,4012,9800,1,1


In [20]:
#Outliers cercas percentilicas

def outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    return df[(df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR))]

In [21]:
OL = []
for i in range(1, 7):
    OL.append(outliers(data, 'bill_amt'+str(i)))

In [22]:
sets = []
for o in OL:
    sets.append(set(o['id']))

outliers = set.intersection(*sets) 

In [ ]:
data[data['id'].isin(outliers)]

,id,limit_bal,sex,education,marriage,age,pay1,pay2,pay3,pay4,pay5,pay6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default payment next month
0,18125,220000,2,3,2,32,-1,-1,-1,-1,-1,-1,209259,192365,184198,171391,178742,164793,7000,15000,6000,10000,10000,7000,0
11,26689,420000,2,2,1,41,-1,-1,-1,-1,-1,-1,373978,389754,366938,361024,354813,344221,33808,14996,13209,12890,11081,16298,1
26,398,500000,1,1,2,30,-1,-1,-1,-1,-1,-1,192257,201402,444462,418237,424244,383145,40000,300018,15000,17000,15000,11100,0
39,10170,270000,2,2,1,27,1,2,-1,-1,-1,-1,253072,247521,253024,258109,243331,247504,0,11000,11000,8900,9000,9200,0
54,17838,250000,2,2,2,29,2,2,2,-1,-1,-1,252354,258503,252087,246649,177663,125960,11783,0,8065,5404,5000,5000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20928,16812,300000,2,2,1,27,-1,-1,-1,-1,-1,-1,286321,283547,578971,293063,296677,230338,20000,12003,10524,11238,8814,9463,1
20946,21387,340000,2,2,2,31,4,3,2,-1,-1,-1,581775,572677,384060,304508,247178,228349,0,0,1000,4320,287982,8007,1
20956,6614,240000,1,2,1,39,1,2,-1,-1,-1,-1,245338,234661,234364,235982,239281,244329,0,10200,10200,8725,9068,9068,0
20973,11410,340000,2,2,1,40,-1,-1,-1,-1,-1,-1,244451,230907,228760,229404,161551,164936,10000,10000,7100,5800,5915,6100,0


In [24]:
data['delay1'] = data['pay1'].apply(lambda x: 1 if x > 0 else 0)
data['delay2'] = data['pay2'].apply(lambda x: 1 if x > 0 else 0)
data['delay3'] = data['pay3'].apply(lambda x: 1 if x > 0 else 0)
data['delay4'] = data['pay4'].apply(lambda x: 1 if x > 0 else 0)
data['delay5'] = data['pay5'].apply(lambda x: 1 if x > 0 else 0)
data['delay6'] = data['pay6'].apply(lambda x: 1 if x > 0 else 0)
data['n_delays'] = data['delay1'] + data['delay2'] + data['delay3'] + data['delay4'] + data['delay5'] + data['delay6']

In [25]:
data = data.drop(['delay1', 'delay2', 'delay3', 'delay4', 'delay5', 'delay6'], axis=1)

In [26]:
data['longest_delay'] = data[['pay1', 'pay2', 'pay3', 'pay4', 'pay5', 'pay6']].max(axis=1)

In [27]:
data['longest_delay'] = data['longest_delay'].apply(lambda x: 0 if x == -1 else x)

In [28]:
data['avg_bill'] = data[['bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6']].mean(axis=1)

In [29]:
data['highest_bill'] = data[['bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6']].max(axis=1)

In [30]:
data['debt/limit'] = data['highest_bill'] / data['limit_bal']

In [31]:
data['avg_pay'] = data[['pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']].mean(axis=1)

In [32]:
data

,id,limit_bal,sex,education,marriage,age,pay1,pay2,pay3,pay4,pay5,pay6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default payment next month,n_delays,longest_delay,avg_bill,highest_bill,debt/limit,avg_pay
0,18125,220000,2,3,2,32,-1,-1,-1,-1,-1,-1,209259,192365,184198,171391,178742,164793,7000,15000,6000,10000,10000,7000,0,0,0,183458.000000,209259,0.951177,9166.666667
1,3983,220000,2,2,1,41,2,-1,-1,2,-1,-1,27094,27819,30363,29579,48933,97187,1465,3009,0,20000,50000,1240,0,2,2,43495.833333,97187,0.441759,12619.000000
2,19251,80000,2,1,2,27,-1,-1,-1,-1,-1,-1,3199,1205,917,917,702,3099,1325,917,0,702,3099,0,0,0,0,1673.166667,3199,0.039988,1007.166667
3,4024,20000,2,2,2,38,1,2,2,4,3,2,10683,12729,14734,14190,13721,13848,2500,2501,0,0,441,1,0,6,4,13317.500000,14734,0.736700,907.166667
4,20610,100000,2,1,2,28,-1,-1,-1,-1,-1,-1,102697,100462,101879,100999,101188,99328,4600,4000,4000,4000,4000,4400,0,0,0,101092.166667,102697,1.026970,4166.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,13019,110000,1,1,2,28,-1,-1,-1,-1,-1,-1,102282,104196,105922,108820,66883,68083,5200,5000,5000,2100,2000,2000,0,0,0,92697.666667,108820,0.989273,3550.000000
20996,27065,420000,2,2,2,25,-1,-1,-1,-1,-1,-1,28206,122212,117835,112705,111357,108306,100000,4013,3822,4086,3800,3600,0,0,0,100103.500000,122212,0.290981,19886.833333
20997,5873,50000,1,2,1,34,-1,-1,-1,-1,-1,-1,48976,47404,46338,14234,19545,20173,2023,3105,3119,11000,909,3000,0,0,0,32778.333333,48976,0.979520,3859.333333
20998,21449,100000,2,1,2,27,-1,-1,-1,-1,-1,-1,-2000,5555,0,0,0,0,7555,0,0,0,0,0,0,0,0,592.500000,5555,0.055550,1259.166667


In [33]:
defs = data[data['default payment next month'] == 1]
ndefs = data[data['default payment next month'] == 0]

In [34]:
defs.describe()

,id,limit_bal,sex,education,marriage,age,pay1,pay2,pay3,pay4,pay5,pay6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default payment next month,n_delays,longest_delay,avg_bill,highest_bill,debt/limit,avg_pay
count,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.0,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000,4555.000000
mean,14989.462349,128608.122942,1.569484,1.881229,1.531943,35.679034,0.455324,0.205049,0.081229,-0.057300,-0.178924,-0.194731,48606.110209,47417.191218,45468.528650,42263.460373,39711.495499,38445.792097,3323.516575,3331.636883,3331.293085,3062.755434,3132.927552,3402.334358,1.0,2.020637,1.456861,43652.096341,54677.264764,0.563892,3264.077314
std,8805.250511,114354.056035,0.495203,0.718160,0.523514,9.665494,1.509279,1.591761,1.579120,1.559678,1.509535,1.489995,74243.060362,72001.220417,68899.053294,64202.887264,61035.974084,60163.870045,8498.842899,10420.102502,12194.008894,8815.479777,11632.956134,13373.945710,0.0,2.121272,1.292717,64571.507196,76908.233943,0.439779,5602.651316
min,10.000000,10000.000000,1.000000,1.000000,1.000000,21.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-6676.000000,-17710.000000,-46127.000000,-50616.000000,-53007.000000,-339603.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,-56043.166667,-6029.000000,-0.018841,0.000000
25%,7237.500000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3036.500000,2928.500000,2526.000000,2307.500000,1650.000000,1307.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,3824.250000,8050.000000,0.083544,800.500000
50%,14964.000000,90000.000000,2.000000,2.000000,2.000000,34.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,20186.000000,20384.000000,20035.000000,19208.000000,18806.000000,18023.000000,1669.000000,1575.000000,1200.000000,1000.000000,1000.000000,1000.000000,1.0,1.000000,2.000000,19915.000000,27418.000000,0.588067,1616.666667
75%,22803.000000,190000.000000,2.000000,2.000000,2.000000,42.000000,2.000000,2.000000,2.000000,2.000000,-1.000000,-1.000000,59549.000000,57405.000000,54925.000000,50464.000000,48042.500000,47574.000000,3446.000000,3310.500000,3000.000000,3000.000000,3000.000000,2995.000000,1.0,4.000000,2.000000,51623.250000,67163.000000,0.970810,3543.666667
max,29998.000000,720000.000000,2.000000,5.000000,3.000000,73.000000,8.000000,7.000000,8.000000,8.000000,8.000000,8.000000,613860.000000,581775.000000,578971.000000,541019.000000,516139.000000,514975.000000,235728.000000,344467.000000,319494.000000,235075.000000,332000.000000,345293.000000,1.0,6.000000,8.000000,521248.333333,613860.000000,3.698100,75805.000000


In [35]:
ndefs.describe()

,id,limit_bal,sex,education,marriage,age,pay1,pay2,pay3,pay4,pay5,pay6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default payment next month,n_delays,longest_delay,avg_bill,highest_bill,debt/limit,avg_pay
count,16445.000000,16445.000000,16445.000000,16445.000000,16445.000000,16445.000000,16445.000000,16445.000000,16445.000000,16445.000000,16445.000000,16445.000000,16445.000000,16445.000000,1.644500e+04,16445.000000,16445.000000,16445.000000,16445.000000,1.644500e+04,16445.000000,16445.000000,16445.00000,16445.000000,16445.0,16445.000000,16445.000000,16445.000000,1.644500e+04,16445.000000,16445.000000
mean,15040.935117,178740.164184,1.615081,1.845059,1.564731,35.453086,-0.656066,-0.734752,-0.735178,-0.779264,-0.818912,-0.800243,52048.586500,49708.569535,4.753991e+04,43770.751110,40694.436242,39206.059289,6296.160718,6.630216e+03,5854.047370,5231.167589,5246.88343,5747.998480,0.0,0.504956,0.474612,45494.718354,6.244001e+04,0.475231,5834.412304
std,8628.089643,132197.109712,0.486591,0.800648,0.520227,9.080832,0.876485,0.892787,0.886660,0.823175,0.752549,0.779230,73807.383292,71060.573733,6.995350e+04,64709.543603,61074.271336,60235.080073,17659.415011,2.475970e+04,19674.924364,16472.725551,16158.11615,19027.613672,0.0,1.149250,0.903137,63336.770882,7.941686e+04,0.426062,11181.118756
min,2.000000,10000.000000,1.000000,1.000000,1.000000,21.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-15308.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-209051.000000,0.000000,0.000000e+00,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,-20913.333333,-4.894000e+03,-0.058000,0.000000
25%,7566.000000,60000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3610.000000,2999.000000,2.632000e+03,2392.000000,1846.000000,1244.000000,1144.000000,1.000000e+03,588.000000,390.000000,350.00000,295.000000,0.0,0.000000,0.000000,5026.166667,1.058400e+04,0.069100,1233.166667
50%,15118.000000,150000.000000,2.000000,2.000000,2.000000,34.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,23018.000000,21484.000000,2.009300e+04,18860.000000,17959.000000,16573.000000,2460.000000,2.245000e+03,2000.000000,1740.000000,1767.00000,1691.000000,0.0,0.000000,0.000000,21353.833333,3.329000e+04,0.386630,2765.000000
75%,22502.000000,250000.000000,2.000000,2.000000,2.000000,41.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,69718.000000,66228.000000,6.220100e+04,56452.000000,51399.000000,49869.000000,5646.000000,5.365000e+03,5000.000000,4625.000000,4626.00000,4571.000000,0.0,0.000000,0.000000,58626.833333,8.282300e+04,0.896938,6198.000000
max,29996.000000,1000000.000000,2.000000,5.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,7.000000,7.000000,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.000000,621000.000000,426529.00000,528666.000000,0.0,6.000000,8.000000,877313.833333,1.664089e+06,6.455300,627344.333333


In [36]:
data['avg pay/debt'] = np.where(data['avg_bill'] == 0, 0, data['avg_pay'] / data['avg_bill']) 

In [37]:
data['small_pay1'] = np.where(data['pay_amt1'] < 0.05*data['bill_amt1'], 1, 0)
data['small_pay2'] = np.where(data['pay_amt2'] < 0.05*data['bill_amt2'], 1, 0)
data['small_pay3'] = np.where(data['pay_amt3'] < 0.05*data['bill_amt3'], 1, 0)
data['small_pay4'] = np.where(data['pay_amt4'] < 0.05*data['bill_amt4'], 1, 0)
data['small_pay5'] = np.where(data['pay_amt5'] < 0.05*data['bill_amt5'], 1, 0)
data['small_pay6'] = np.where(data['pay_amt6'] < 0.05*data['bill_amt6'], 1, 0)

data['n_small_pays'] = data['small_pay1'] + data['small_pay2'] + data['small_pay3'] + data['small_pay4'] + data['small_pay5'] + data['small_pay6']

In [38]:
data = data.drop(['small_pay1', 'small_pay2', 'small_pay3', 'small_pay4', 'small_pay5', 'small_pay6'], axis=1)

In [39]:
data

,id,limit_bal,sex,education,marriage,age,pay1,pay2,pay3,pay4,pay5,pay6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default payment next month,n_delays,longest_delay,avg_bill,highest_bill,debt/limit,avg_pay,avg pay/debt,n_small_pays
0,18125,220000,2,3,2,32,-1,-1,-1,-1,-1,-1,209259,192365,184198,171391,178742,164793,7000,15000,6000,10000,10000,7000,0,0,0,183458.000000,209259,0.951177,9166.666667,0.049966,3
1,3983,220000,2,2,1,41,2,-1,-1,2,-1,-1,27094,27819,30363,29579,48933,97187,1465,3009,0,20000,50000,1240,0,2,2,43495.833333,97187,0.441759,12619.000000,0.290120,2
2,19251,80000,2,1,2,27,-1,-1,-1,-1,-1,-1,3199,1205,917,917,702,3099,1325,917,0,702,3099,0,0,0,0,1673.166667,3199,0.039988,1007.166667,0.601952,2
3,4024,20000,2,2,2,38,1,2,2,4,3,2,10683,12729,14734,14190,13721,13848,2500,2501,0,0,441,1,0,6,4,13317.500000,14734,0.736700,907.166667,0.068118,4
4,20610,100000,2,1,2,28,-1,-1,-1,-1,-1,-1,102697,100462,101879,100999,101188,99328,4600,4000,4000,4000,4000,4400,0,0,0,101092.166667,102697,1.026970,4166.666667,0.041217,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,13019,110000,1,1,2,28,-1,-1,-1,-1,-1,-1,102282,104196,105922,108820,66883,68083,5200,5000,5000,2100,2000,2000,0,0,0,92697.666667,108820,0.989273,3550.000000,0.038297,5
20996,27065,420000,2,2,2,25,-1,-1,-1,-1,-1,-1,28206,122212,117835,112705,111357,108306,100000,4013,3822,4086,3800,3600,0,0,0,100103.500000,122212,0.290981,19886.833333,0.198663,5
20997,5873,50000,1,2,1,34,-1,-1,-1,-1,-1,-1,48976,47404,46338,14234,19545,20173,2023,3105,3119,11000,909,3000,0,0,0,32778.333333,48976,0.979520,3859.333333,0.117740,2
20998,21449,100000,2,1,2,27,-1,-1,-1,-1,-1,-1,-2000,5555,0,0,0,0,7555,0,0,0,0,0,0,0,0,592.500000,5555,0.055550,1259.166667,2.125176,1


In [40]:
for i in range(1, 7):
    data['pay'+str(i)] = data['pay'+str(i)].replace({-1:0, 1:1, 2:1, 3:1, 4:1, 5:1, 6:1, 7:1, 8:1})

In [41]:
data

,id,limit_bal,sex,education,marriage,age,pay1,pay2,pay3,pay4,pay5,pay6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default payment next month,n_delays,longest_delay,avg_bill,highest_bill,debt/limit,avg_pay,avg pay/debt,n_small_pays
0,18125,220000,2,3,2,32,0,0,0,0,0,0,209259,192365,184198,171391,178742,164793,7000,15000,6000,10000,10000,7000,0,0,0,183458.000000,209259,0.951177,9166.666667,0.049966,3
1,3983,220000,2,2,1,41,1,0,0,1,0,0,27094,27819,30363,29579,48933,97187,1465,3009,0,20000,50000,1240,0,2,2,43495.833333,97187,0.441759,12619.000000,0.290120,2
2,19251,80000,2,1,2,27,0,0,0,0,0,0,3199,1205,917,917,702,3099,1325,917,0,702,3099,0,0,0,0,1673.166667,3199,0.039988,1007.166667,0.601952,2
3,4024,20000,2,2,2,38,1,1,1,1,1,1,10683,12729,14734,14190,13721,13848,2500,2501,0,0,441,1,0,6,4,13317.500000,14734,0.736700,907.166667,0.068118,4
4,20610,100000,2,1,2,28,0,0,0,0,0,0,102697,100462,101879,100999,101188,99328,4600,4000,4000,4000,4000,4400,0,0,0,101092.166667,102697,1.026970,4166.666667,0.041217,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,13019,110000,1,1,2,28,0,0,0,0,0,0,102282,104196,105922,108820,66883,68083,5200,5000,5000,2100,2000,2000,0,0,0,92697.666667,108820,0.989273,3550.000000,0.038297,5
20996,27065,420000,2,2,2,25,0,0,0,0,0,0,28206,122212,117835,112705,111357,108306,100000,4013,3822,4086,3800,3600,0,0,0,100103.500000,122212,0.290981,19886.833333,0.198663,5
20997,5873,50000,1,2,1,34,0,0,0,0,0,0,48976,47404,46338,14234,19545,20173,2023,3105,3119,11000,909,3000,0,0,0,32778.333333,48976,0.979520,3859.333333,0.117740,2
20998,21449,100000,2,1,2,27,0,0,0,0,0,0,-2000,5555,0,0,0,0,7555,0,0,0,0,0,0,0,0,592.500000,5555,0.055550,1259.166667,2.125176,1


In [42]:
#Dimensionality Reduction

X = data.drop(['id', 'default payment next month'], axis=1)
y = data['default payment next month']

In [43]:
#SelectKBest

skb = SelectKBest(f_classif, k=10)
skb.fit(X, y)

SelectKBest()

In [44]:
best = pd.DataFrame({'Feature': X.columns, 'Scores': skb.scores_})

In [45]:
best.sort_values(by='Scores', ascending=False)

,Feature,Scores
23,n_delays,4076.352092
24,longest_delay,3437.401852
5,pay1,3378.344066
6,pay2,2778.257917
7,pay3,2082.996564
8,pay4,1781.420281
9,pay5,1573.141027
10,pay6,1343.795704
0,limit_bal,542.589782
28,avg_pay,225.054366


In [46]:
top10 = best.sort_values(by='Scores', ascending=False).head(10)['Feature'].values

In [47]:
#VarClusHi

vc = VarClusHi(X, maxeigval2=1, maxclus=6)

In [48]:
vc.varclus()

In [49]:
#Ver clusters

vc.rsquare

,Cluster,Variable,RS_Own,RS_NC,RS_Ratio
0,0,bill_amt1,0.885998,0.075186,1.232704e-01
1,0,bill_amt2,0.917943,0.091669,9.033786e-02
2,0,bill_amt3,0.915923,0.132497,9.691856e-02
3,0,bill_amt4,0.913483,0.139389,1.005303e-01
4,0,bill_amt5,0.886844,0.146056,1.325103e-01
5,0,bill_amt6,0.847861,0.146037,1.781568e-01
6,0,avg_bill,0.987636,0.130166,1.421378e-02
7,0,highest_bill,0.900608,0.241415,1.310229e-01
8,0,debt/limit,0.320142,0.023657,6.963309e-01
9,0,n_small_pays,0.382306,0.055415,6.539310e-01


In [50]:
#Escalamiento 

scx = StandardScaler()
scy = StandardScaler()

mmx = MinMaxScaler()
mmy = MinMaxScaler()

rsx = RobustScaler()
rsy = RobustScaler()

X1 = scx.fit_transform(X)
y1 = scy.fit_transform(y.values.reshape(-1, 1))

X2 = mmx.fit_transform(X)
y2 = mmy.fit_transform(y.values.reshape(-1, 1))

X3 = rsx.fit_transform(X)
y3 = rsy.fit_transform(y.values.reshape(-1, 1))

In [51]:
#Escalamiento Xtop

Xtop = X[top10]

Xtop1 = scx.fit_transform(Xtop)
Xtop2 = mmx.fit_transform(Xtop)
Xtop3 = rsx.fit_transform(Xtop)

In [52]:
#Logistic Regression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

lr = LogisticRegression()

lr.fit(X_train, y_train)

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

In [53]:
#scores train

lr.score(X_train, y_train)

0.7846984126984127

In [54]:
#cross validation

lrc = cross_val_score(lr, X_train, y_train, cv=5)

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

In [55]:
lrc.mean(), lrc.std()

(0.7847619047619048, 0.0003477603539715247)

In [56]:
#Validacion en test

lr.score(X_test, y_test)

0.7754285714285715

In [57]:
#Métricas de performance

y_pred = lr.predict(X_test)

print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      1.00      0.87      4077
           1       0.12      0.00      0.00      1173

    accuracy                           0.78      5250
   macro avg       0.45      0.50      0.44      5250
weighted avg       0.63      0.78      0.68      5250

[[4070    7]
 [1172    1]]


In [58]:
roc_auc_score(y_test, y_pred)

0.4995677830910974

In [59]:
#Logistic Regression con escalamiento

#StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.25, random_state=42)

lr1 = LogisticRegression()

lr1.fit(X_train, y_train)

LogisticRegression()

In [60]:
#scores train

lr1.score(X_train, y_train)

0.8128888888888889

In [61]:
#cross validation

lrc1 = cross_val_score(lr1, X_train, y_train, cv=5)

In [62]:
lrc1

array([0.81619048, 0.80888889, 0.81206349, 0.81269841, 0.80984127])

In [63]:
lrc1.mean(), lrc1.std()

(0.811936507936508, 0.0025444399885097245)

In [64]:
#Validacion en test

lr1.score(X_test, y_test)

0.8097142857142857

In [65]:
#métricas de performance

y_pred = lr1.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.83      0.95      0.89      4077
           1       0.66      0.31      0.42      1173

    accuracy                           0.81      5250
   macro avg       0.74      0.63      0.65      5250
weighted avg       0.79      0.81      0.78      5250



In [66]:
#matriz de confusión

print(confusion_matrix(y_test, y_pred))

[[3884  193]
 [ 806  367]]


In [67]:
roc_auc_score(y_test, y_pred)

0.6327671229095664

In [68]:
#minmaxscaler

X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.25, random_state=42)

lr2 = LogisticRegression()

lr2.fit(X_train, y_train)

LogisticRegression()

In [69]:
#scores train

lr2.score(X_train, y_train)

0.8128888888888889

In [70]:
#cross validation

lrc2 = cross_val_score(lr2, X_train, y_train, cv=5)

lrc2

array([0.81619048, 0.80888889, 0.81206349, 0.81269841, 0.80984127])

In [71]:
lrc2.mean(), lrc2.std()

(0.811936507936508, 0.0025444399885097245)

In [72]:
#Validacion en test

lr2.score(X_test, y_test)

0.8097142857142857

In [73]:
#métricas de performance

y_pred = lr2.predict(X_test)

print(classification_report(y_test, y_pred))

#matriz de confusión

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.95      0.89      4077
           1       0.66      0.31      0.42      1173

    accuracy                           0.81      5250
   macro avg       0.74      0.63      0.65      5250
weighted avg       0.79      0.81      0.78      5250

[[3884  193]
 [ 806  367]]


In [74]:
roc_auc_score(y_test, y_pred)

0.6327671229095664

In [75]:
#robuscaler

X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.25, random_state=42)

lr3 = LogisticRegression()

lr3.fit(X_train, y_train)

LogisticRegression()

In [76]:
#scores train

lr3.score(X_train, y_train)

0.8135238095238095

In [77]:
#cross validation

lrc3 = cross_val_score(lr3, X_train, y_train, cv=5)

lrc3

array([0.81460317, 0.81079365, 0.81492063, 0.81333333, 0.81047619])

In [78]:
lrc3.mean(), lrc3.std()

(0.8128253968253969, 0.0018684366945475631)

In [79]:
#Validacion en test

lr3.score(X_test, y_test)

0.8099047619047619

In [80]:
#métricas de performance

y_pred = lr3.predict(X_test)

print(classification_report(y_test, y_pred))

#matriz de confusión

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.95      0.89      4077
           1       0.66      0.31      0.43      1173

    accuracy                           0.81      5250
   macro avg       0.74      0.63      0.66      5250
weighted avg       0.79      0.81      0.78      5250

[[3883  194]
 [ 804  369]]


In [81]:
roc_auc_score(y_test, y_pred)

0.6334969986330905

In [82]:
#Logistic Regression con top10

X_train, X_test, y_train, y_test = train_test_split(Xtop, y, test_size=0.25, random_state=42)

lr4 = LogisticRegression()

lr4.fit(X_train, y_train)

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

In [83]:
#scores train

lr4.score(X_train, y_train)

0.8093333333333333

In [84]:
#cross validation

lrc4 = cross_val_score(lr4, X_train, y_train, cv=5)

lrc4

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

array([0.81238095, 0.80888889, 0.80730159, 0.81174603, 0.80857143])

In [85]:
lrc4.mean(), lrc4.std()

(0.8097777777777777, 0.0019507671740618831)

In [86]:
#Validacion en test

lr4.score(X_test, y_test)

0.8076190476190476

In [87]:
#métricas de performance

y_pred = lr4.predict(X_test)

print(classification_report(y_test, y_pred))

#matriz de confusión

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.96      0.89      4077
           1       0.66      0.29      0.40      1173

    accuracy                           0.81      5250
   macro avg       0.74      0.62      0.64      5250
weighted avg       0.79      0.81      0.78      5250

[[3901  176]
 [ 834  339]]


In [88]:
roc_auc_score(y_test, y_pred)

0.6229167803666881

In [89]:
#LDA

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

lda = LinearDiscriminantAnalysis()

lda.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [90]:
#scores train

lda.score(X_train, y_train)

0.811047619047619

In [91]:
#cross validation

ldac = cross_val_score(lda, X_train, y_train, cv=5)

ldac

array([0.81333333, 0.80666667, 0.81301587, 0.81079365, 0.80507937])

In [92]:
ldac.mean(), ldac.std()

(0.8097777777777779, 0.0033440492907336437)

In [93]:
#Validacion en test

lda.score(X_test, y_test)

0.8102857142857143

In [94]:
#métricas de performance

y_pred = lda.predict(X_test)

print(classification_report(y_test, y_pred))

#matriz de confusión

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.93      0.88      4077
           1       0.62      0.38      0.47      1173

    accuracy                           0.81      5250
   macro avg       0.73      0.66      0.68      5250
weighted avg       0.79      0.81      0.79      5250

[[3810  267]
 [ 729  444]]


In [95]:
#LDA con escalamiento

#StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.25, random_state=42)

lda1 = LinearDiscriminantAnalysis()

lda1.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [96]:
#scores train

lda1.score(X_train, y_train)

0.811047619047619

In [97]:
#cross validation

ldac1 = cross_val_score(lda1, X_train, y_train, cv=5)

ldac1

array([0.81333333, 0.80666667, 0.81301587, 0.81079365, 0.80507937])

In [98]:
ldac1.mean(), ldac1.std()  

(0.8097777777777779, 0.0033440492907336437)

In [99]:
#Validacion en test

lda1.score(X_test, y_test)

0.8102857142857143

In [100]:
#métricas de performance

y_pred = lda1.predict(X_test)

print(classification_report(y_test, y_pred))

#matriz de confusión

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.93      0.88      4077
           1       0.62      0.38      0.47      1173

    accuracy                           0.81      5250
   macro avg       0.73      0.66      0.68      5250
weighted avg       0.79      0.81      0.79      5250

[[3810  267]
 [ 729  444]]


In [101]:
#MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.25, random_state=42)

lda2 = LinearDiscriminantAnalysis()

lda2.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [102]:
#scores train

lda2.score(X_train, y_train)

0.811047619047619

In [103]:
#cross validation

ldac2 = cross_val_score(lda2, X_train, y_train, cv=5)

ldac2

ldac2.mean(), ldac2.std()

(0.8097777777777779, 0.0033440492907336437)

In [104]:
#Validacion en test

lda2.score(X_test, y_test)

0.8102857142857143

In [105]:
#métricas de performance

y_pred = lda2.predict(X_test)

print(classification_report(y_test, y_pred))

#matriz de confusión

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.93      0.88      4077
           1       0.62      0.38      0.47      1173

    accuracy                           0.81      5250
   macro avg       0.73      0.66      0.68      5250
weighted avg       0.79      0.81      0.79      5250

[[3810  267]
 [ 729  444]]


In [106]:
#RobustScaler

X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.25, random_state=42)

lda3 = LinearDiscriminantAnalysis()

lda3.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [107]:
#scores train

lda3.score(X_train, y_train)

0.811047619047619

In [108]:
#cross validation

ldac3 = cross_val_score(lda3, X_train, y_train, cv=5)

ldac3

array([0.81333333, 0.80666667, 0.81301587, 0.81079365, 0.80507937])

In [109]:
ldac3.mean(), ldac3.std()

(0.8097777777777779, 0.0033440492907336437)

In [110]:
#Validacion en test

lda3.score(X_test, y_test)

0.8102857142857143

In [111]:
#métricas de performance

y_pred = lda3.predict(X_test)

print(classification_report(y_test, y_pred))

#matriz de confusión

print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.84      0.93      0.88      4077
           1       0.62      0.38      0.47      1173

    accuracy                           0.81      5250
   macro avg       0.73      0.66      0.68      5250
weighted avg       0.79      0.81      0.79      5250

[[3810  267]
 [ 729  444]]


In [112]:
#LDA con top10

X_train, X_test, y_train, y_test = train_test_split(Xtop, y, test_size=0.25, random_state=42)

lda4 = LinearDiscriminantAnalysis()

lda4.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [113]:
#scores train

lda4.score(X_train, y_train)

0.8100952380952381

In [114]:
#cross validation

ldac4 = cross_val_score(lda4, X_train, y_train, cv=5)

ldac4

ldac4.mean(), ldac4.std()



(0.8093968253968253, 0.0028223632724943388)

In [115]:
#validacion en test

lda4.score(X_test, y_test)

0.808

In [116]:
#métricas de performance

y_pred = lda4.predict(X_test)

print(classification_report(y_test, y_pred))

#matriz de confusión

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.93      0.88      4077
           1       0.62      0.37      0.47      1173

    accuracy                           0.81      5250
   macro avg       0.73      0.65      0.67      5250
weighted avg       0.79      0.81      0.79      5250

[[3803  274]
 [ 734  439]]


In [117]:
#SVM

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

svm = SVC()

svm.fit(X_train, y_train)

SVC()

In [118]:
#scores train

svm.score(X_train, y_train)

0.7853333333333333

In [119]:
#cross validation

svmc = cross_val_score(svm, X_train, y_train, cv=5, n_jobs=-1)

svmc

array([0.78507937, 0.78507937, 0.78539683, 0.78539683, 0.78539683])

In [120]:
#svm top10

X_train, X_test, y_train, y_test = train_test_split(Xtop, y, test_size=0.25, random_state=42)

svm4 = SVC()

svm4.fit(X_train, y_train)

SVC()

In [121]:
#scores train

svm4.score(X_train, y_train)

0.7852698412698412

In [122]:
#cross validation

svm4c = cross_val_score(svm4, X_train, y_train, cv=5, n_jobs=-1)

svm4c

array([0.78507937, 0.78507937, 0.78539683, 0.78539683, 0.78539683])

In [123]:
svm4c.mean(), svm4c.std()

(0.7852698412698413, 0.0001555231582719799)

In [124]:
#Validacion en test

svm4.score(X_test, y_test)

0.7765714285714286

In [125]:
#métricas de performance

y_pred = svm4.predict(X_test)

print(classification_report(y_test, y_pred))

#matriz de confusión

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      1.00      0.87      4077
           1       0.00      0.00      0.00      1173

    accuracy                           0.78      5250
   macro avg       0.39      0.50      0.44      5250
weighted avg       0.60      0.78      0.68      5250

[[4077    0]
 [1173    0]]


/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [126]:
#KNN

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

knn = KNeighborsClassifier()

knn.fit(X_train, y_train)

KNeighborsClassifier()

In [127]:
#scores train

knn.score(X_train, y_train)

0.8185396825396826

In [128]:
#cross validation

knnc = cross_val_score(knn, X_train, y_train, cv=5, n_jobs=-1)

knnc

array([0.76349206, 0.76      , 0.75174603, 0.75746032, 0.75809524])

In [129]:
knnc.mean(), knnc.std()

(0.7581587301587301, 0.003832733529596585)

In [130]:
#validacion en test

knn.score(X_test, y_test)

0.7518095238095238

In [131]:
#métricas de performance

y_pred = knn.predict(X_test)

print(classification_report(y_test, y_pred))

#matriz de confusión

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.92      0.85      4077
           1       0.38      0.17      0.23      1173

    accuracy                           0.75      5250
   macro avg       0.58      0.54      0.54      5250
weighted avg       0.70      0.75      0.71      5250

[[3751  326]
 [ 977  196]]


In [132]:
#KNN con escalamiento

#StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.25, random_state=42)

knn1 = KNeighborsClassifier()

knn1.fit(X_train, y_train)

KNeighborsClassifier()

In [133]:
#scores train

knn1.score(X_train, y_train)

0.8436825396825397

In [134]:
#cross validation

knn1c = cross_val_score(knn1, X_train, y_train, cv=5, n_jobs=-1)

knn1c

array([0.78857143, 0.79492063, 0.78698413, 0.79079365, 0.78952381])

In [135]:
#Validacion en test

knn1.score(X_test, y_test)

0.7895238095238095

In [136]:
#métricas de performance

y_pred = knn1.predict(X_test)

print(classification_report(y_test, y_pred))

#matriz de confusión

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.91      0.87      4077
           1       0.54      0.36      0.43      1173

    accuracy                           0.79      5250
   macro avg       0.69      0.64      0.65      5250
weighted avg       0.77      0.79      0.77      5250

[[3721  356]
 [ 749  424]]


In [137]:
#MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.25, random_state=42)

knn2 = KNeighborsClassifier()

knn2.fit(X_train, y_train)

KNeighborsClassifier()

In [138]:
#scores train

knn2.score(X_train, y_train)

0.843047619047619

In [139]:
#cross validation

knn2c = cross_val_score(knn2, X_train, y_train, cv=5, n_jobs=-1)

knn2c

array([0.78666667, 0.78793651, 0.78634921, 0.78920635, 0.78444444])

In [140]:
#Validacion en test

knn2.score(X_test, y_test)

0.7845714285714286

In [141]:
#métricas de performance

y_pred = knn2.predict(X_test)

print(classification_report(y_test, y_pred))

#matriz de confusión

print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.83      0.91      0.87      4077
           1       0.53      0.34      0.42      1173

    accuracy                           0.78      5250
   macro avg       0.68      0.63      0.64      5250
weighted avg       0.76      0.78      0.77      5250

[[3717  360]
 [ 771  402]]


In [142]:
#RobustScaler

X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.25, random_state=42)

knn3 = KNeighborsClassifier()

knn3.fit(X_train, y_train)

KNeighborsClassifier()

In [143]:
#scores train

knn3.score(X_train, y_train)

0.8466666666666667

In [144]:
#cross validation

knn3c = cross_val_score(knn3, X_train, y_train, cv=5, n_jobs=-1)

knn3c

array([0.79746032, 0.7952381 , 0.79269841, 0.79301587, 0.78412698])

In [145]:
knn3c.mean(), knn3c.std()

(0.7925079365079365, 0.00452801252817153)

In [146]:
#Validacion en test

knn3.score(X_test, y_test)

0.7876190476190477

In [147]:
#Knn con top10

X_train, X_test, y_train, y_test = train_test_split(Xtop, y, test_size=0.25, random_state=42)

knn4 = KNeighborsClassifier()

knn4.fit(X_train, y_train)


KNeighborsClassifier()

In [148]:
#scores train

knn4.score(X_train, y_train)


0.8167619047619048

In [149]:
#cross validation

knn4c = cross_val_score(knn4, X_train, y_train, cv=5, n_jobs=-1)

knn4c

array([0.74825397, 0.76222222, 0.75111111, 0.75111111, 0.75809524])

In [150]:
#Validacion en test

knn4.score(X_test, y_test)

0.7497142857142857

In [151]:
datatest = pd.read_csv('./test_p3.csv')

In [152]:
datatest.columns = ['id', 'limit_bal','sex', 'education', 'marriage', 'age', 'pay1', 'pay2', 'pay3', 'pay4', 'pay5', 'pay6', 'bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6', 'pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']

In [153]:
datatest['education'] = datatest['education'].replace({0:5, 6:5})

In [154]:
datatest['marriage'] = datatest['marriage'].replace({0:3})

In [155]:
for i in range(1, 7):
    datatest['pay'+str(i)] = datatest['pay'+str(i)].replace({-2:-1, 0:-1})

In [156]:
datatest['delay1'] = datatest['pay1'].apply(lambda x: 1 if x > 0 else 0)
datatest['delay2'] = datatest['pay2'].apply(lambda x: 1 if x > 0 else 0)
datatest['delay3'] = datatest['pay3'].apply(lambda x: 1 if x > 0 else 0)
datatest['delay4'] = datatest['pay4'].apply(lambda x: 1 if x > 0 else 0)
datatest['delay5'] = datatest['pay5'].apply(lambda x: 1 if x > 0 else 0)
datatest['delay6'] = datatest['pay6'].apply(lambda x: 1 if x > 0 else 0)

datatest['n_delays'] = datatest['delay1'] + datatest['delay2'] + datatest['delay3'] + datatest['delay4'] + datatest['delay5'] + datatest['delay6']

In [157]:
datatest = datatest.drop(['delay1', 'delay2', 'delay3', 'delay4', 'delay5', 'delay6'], axis=1)

In [158]:
datatest['longest_delay'] = datatest[['pay1', 'pay2', 'pay3', 'pay4', 'pay5', 'pay6']].max(axis=1)

In [159]:
datatest['avg_bill'] = datatest[['bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6']].mean(axis=1)

In [160]:
datatest['highest_bill'] = datatest[['bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6']].max(axis=1)

In [161]:
datatest['debt/limit'] = datatest['highest_bill'] / datatest['limit_bal']

In [162]:
datatest['avg_pay'] = datatest[['pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']].mean(axis=1)

In [163]:
datatest['avg pay/debt'] = np.where(datatest['avg_bill'] == 0, 0, datatest['avg_pay'] / datatest['avg_bill'])

In [164]:
datatest['small_pay1'] = np.where(datatest['pay_amt1'] < 0.05*datatest['bill_amt1'], 1, 0)
datatest['small_pay2'] = np.where(datatest['pay_amt2'] < 0.05*datatest['bill_amt2'], 1, 0)
datatest['small_pay3'] = np.where(datatest['pay_amt3'] < 0.05*datatest['bill_amt3'], 1, 0)
datatest['small_pay4'] = np.where(datatest['pay_amt4'] < 0.05*datatest['bill_amt4'], 1, 0)
datatest['small_pay5'] = np.where(datatest['pay_amt5'] < 0.05*datatest['bill_amt5'], 1, 0)
datatest['small_pay6'] = np.where(datatest['pay_amt6'] < 0.05*datatest['bill_amt6'], 1, 0)

datatest['n_small_pays'] = datatest['small_pay1'] + datatest['small_pay2'] + datatest['small_pay3'] + datatest['small_pay4'] + datatest['small_pay5'] + datatest['small_pay6']

In [165]:
datatest = datatest.drop(['small_pay1', 'small_pay2', 'small_pay3', 'small_pay4', 'small_pay5', 'small_pay6'], axis=1)

In [166]:
for i in range(1, 7):
    datatest['pay'+str(i)] = datatest['pay'+str(i)].replace({-1:0, 1:1, 2:1, 3:1, 4:1, 5:1, 6:1, 7:1, 8:1})

In [167]:
datatest

,id,limit_bal,sex,education,marriage,age,pay1,pay2,pay3,pay4,pay5,pay6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,n_delays,longest_delay,avg_bill,highest_bill,debt/limit,avg_pay,avg pay/debt,n_small_pays
0,21256,320000,2,2,2,50,0,0,0,0,0,0,5714,7870,342,-6,1252,0,3000,342,2,1500,0,10701,0,-1,2528.666667,7870,0.024594,2590.833333,1.024585,3
1,14589,380000,2,1,1,43,0,0,0,0,0,0,18866,20840,188,2172,5173,5446,20884,188,2172,5173,5446,2045,0,-1,8780.833333,20840,0.054842,5984.666667,0.681560,1
2,19988,180000,2,2,1,47,0,0,0,0,0,0,107,825,7609,825,1650,825,825,7609,825,1650,0,8214,0,-1,1973.500000,7609,0.042272,3187.166667,1.614982,1
3,21287,110000,1,2,1,30,1,1,1,1,1,1,88168,90585,89555,87410,85576,84021,7300,3800,3000,3100,3000,3000,6,2,87552.500000,90585,0.823500,3866.666667,0.044164,5
4,12418,40000,2,2,1,49,0,0,0,0,0,0,36159,38241,39121,40400,40000,40000,3000,1800,2000,0,0,0,0,-1,38986.833333,40400,1.010000,1133.333333,0.029070,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,12905,50000,2,5,1,28,0,0,0,0,0,0,9014,9608,12025,16025,0,0,3000,3000,4000,0,0,0,0,-1,7778.666667,16025,0.320500,1666.666667,0.214261,1
8996,3995,170000,1,2,1,36,1,1,0,0,0,0,158954,139482,139869,139956,141431,149946,7,6513,6548,5300,11001,0,2,2,144939.666667,158954,0.935024,4894.833333,0.033772,5
8997,25540,120000,2,3,1,63,0,0,0,0,0,0,49394,51261,51333,51174,44680,43760,3000,2000,2000,2000,2000,2000,0,-1,48600.333333,51333,0.427775,2166.666667,0.044581,5
8998,22474,360000,1,1,1,48,0,0,0,0,0,0,5518,2532,5504,-4,-4,881,2548,5536,0,0,885,1595,0,-1,2404.500000,5518,0.015328,1760.666667,0.732238,1


In [168]:
datatest['longest_delay'] = datatest['longest_delay'].apply(lambda x: 0 if x == -1 else x)

In [169]:
Xt = datatest.drop('id', axis=1)

In [170]:
#Best model Logistic Regression RobustScaler

X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.25, random_state=42)

lr3 = LogisticRegression()

lr3.fit(X_train, y_train)

LogisticRegression()

In [171]:
#scores train

lr3.score(X_train, y_train)

0.8135238095238095

In [172]:
#cross validation

lrc3 = cross_val_score(lr3, X_train, y_train, cv=5)

lrc3

array([0.81460317, 0.81079365, 0.81492063, 0.81333333, 0.81047619])

In [173]:
lrc3.mean(), lrc3.std()

(0.8128253968253969, 0.0018684366945475631)

In [174]:
#Validacion en test

lr3.score(X_test, y_test)

0.8099047619047619

In [175]:
#Datos de test
#RObustScaler

rbst = RobustScaler()
Xt = rbst.fit_transform(Xt)

In [176]:
#Predict con lr3

y_pred = lr3.predict(Xt)

y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [177]:
#Probabilidad de 1: default

y_prob = lr3.predict_proba(Xt)[:, 1]

df = pd.DataFrame({'id': datatest['id'], 'prob def': y_prob})

In [178]:
df

,id,prob def
0,21256,0.081863
1,14589,0.074895
2,19988,0.119654
3,21287,0.672437
4,12418,0.152103
...,...,...
8995,12905,0.116537
8996,3995,0.446401
8997,25540,0.131317
8998,22474,0.104861


In [179]:
df.columns = ['ID', 'y_hat']

In [180]:
#roc auc score

roc_auc_score(y_test, lr3.predict_proba(X_test)[:, 1])

0.7591042090231919

In [181]:
#Mejorar el modelo con RandomizedSearchCV

#Logistic Regression, RobustScaler 

X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.25, random_state=42)

lr3 = LogisticRegression()

param_grid = {'penalty': ['l1', 'l2', 'elasticnet'], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

In [182]:
gs = GridSearchCV(lr3, param_grid, cv=5, n_jobs=-1, scoring='roc_auc')

In [183]:
gs.fit(X_train, y_train)

/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/josemariareyesfigueroa/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='roc_auc')

In [184]:
gs.best_params_

{'penalty': 'l1', 'solver': 'liblinear'}

In [185]:
gs.best_score_ 

0.7629566708863538

In [186]:
#Predict un modelo mejorado

y_pred = gs.predict(Xt)

y_prob = gs.predict_proba(Xt)[:, 1]

df = pd.DataFrame({'id': datatest['id'], 'prob def': y_prob})

df

,id,prob def
0,21256,0.082203
1,14589,0.075340
2,19988,0.119578
3,21287,0.672022
4,12418,0.152278
...,...,...
8995,12905,0.116755
8996,3995,0.447904
8997,25540,0.131571
8998,22474,0.104582


In [187]:
pd.to_pickle(gs, './gs.pkl')

In [283]:
df.columns = ['ID', 'y_hat']

In [284]:
df.to_csv('CDD_G26_M2_P3_JoseMara_Reyes_Figueroa.csv', index=False)

In [288]:
data

,id,limit_bal,sex,education,marriage,age,pay1,pay2,pay3,pay4,pay5,pay6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default payment next month,n_delays,longest_delay,avg_bill,highest_bill,debt/limit,avg_pay,avg pay/debt,n_small_pays
0,18125,220000,2,3,2,32,0,0,0,0,0,0,209259,192365,184198,171391,178742,164793,7000,15000,6000,10000,10000,7000,0,0,0,183458.000000,209259,0.951177,9166.666667,0.049966,3
1,3983,220000,2,2,1,41,1,0,0,1,0,0,27094,27819,30363,29579,48933,97187,1465,3009,0,20000,50000,1240,0,2,2,43495.833333,97187,0.441759,12619.000000,0.290120,2
2,19251,80000,2,1,2,27,0,0,0,0,0,0,3199,1205,917,917,702,3099,1325,917,0,702,3099,0,0,0,0,1673.166667,3199,0.039988,1007.166667,0.601952,2
3,4024,20000,2,2,2,38,1,1,1,1,1,1,10683,12729,14734,14190,13721,13848,2500,2501,0,0,441,1,0,6,4,13317.500000,14734,0.736700,907.166667,0.068118,4
4,20610,100000,2,1,2,28,0,0,0,0,0,0,102697,100462,101879,100999,101188,99328,4600,4000,4000,4000,4000,4400,0,0,0,101092.166667,102697,1.026970,4166.666667,0.041217,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,13019,110000,1,1,2,28,0,0,0,0,0,0,102282,104196,105922,108820,66883,68083,5200,5000,5000,2100,2000,2000,0,0,0,92697.666667,108820,0.989273,3550.000000,0.038297,5
20996,27065,420000,2,2,2,25,0,0,0,0,0,0,28206,122212,117835,112705,111357,108306,100000,4013,3822,4086,3800,3600,0,0,0,100103.500000,122212,0.290981,19886.833333,0.198663,5
20997,5873,50000,1,2,1,34,0,0,0,0,0,0,48976,47404,46338,14234,19545,20173,2023,3105,3119,11000,909,3000,0,0,0,32778.333333,48976,0.979520,3859.333333,0.117740,2
20998,21449,100000,2,1,2,27,0,0,0,0,0,0,-2000,5555,0,0,0,0,7555,0,0,0,0,0,0,0,0,592.500000,5555,0.055550,1259.166667,2.125176,1


In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np

# Leer el archivo CSV
datatest = pd.read_csv('ruta/datatest.csv')

# Aplicar las transformaciones previas
datatest.columns = ['id', 'limit_bal','sex', 'education', 'marriage', 'age', 'pay1', 'pay2', 'pay3', 'pay4', 'pay5', 'pay6', 
                    'bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6', 
                    'pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']

datatest['education'] = datatest['education'].replace({0:5, 6:5})
datatest['marriage'] = datatest['marriage'].replace({0:3})

# Reemplazar valores de pay1 a pay6
for i in range(1, 7):
    datatest['pay'+str(i)] = datatest['pay'+str(i)].replace({-2:-1, 0:-1})

# Crear columnas de "delay" y calcular el número de retrasos
for i in range(1, 7):
    datatest['delay'+str(i)] = datatest['pay'+str(i)].apply(lambda x: 1 if x > 0 else 0)

datatest['n_delays'] = datatest[['delay1', 'delay2', 'delay3', 'delay4', 'delay5', 'delay6']].sum(axis=1)

# Eliminar las columnas de "delay"
datatest = datatest.drop(['delay1', 'delay2', 'delay3', 'delay4', 'delay5', 'delay6'], axis=1)

# Calcular la demora más larga, el promedio de las facturas y otros valores
datatest['longest_delay'] = datatest[['pay1', 'pay2', 'pay3', 'pay4', 'pay5', 'pay6']].max(axis=1)
datatest['avg_bill'] = datatest[['bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6']].mean(axis=1)
datatest['highest_bill'] = datatest[['bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6']].max(axis=1)
datatest['debt/limit'] = datatest['highest_bill'] / datatest['limit_bal']
datatest['avg_pay'] = datatest[['pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']].mean(axis=1)
datatest['avg pay/debt'] = np.where(datatest['avg_bill'] == 0, 0, datatest['avg_pay'] / datatest['avg_bill'])

# Crear columnas de "small_pay" y calcular el número de pagos pequeños
for i in range(1, 7):
    datatest['small_pay'+str(i)] = np.where(datatest['pay_amt'+str(i)] < 0.05*datatest['bill_amt'+str(i)], 1, 0)

datatest['n_small_pays'] = datatest[['small_pay1', 'small_pay2', 'small_pay3', 'small_pay4', 'small_pay5', 'small_pay6']].sum(axis=1)

# Eliminar las columnas de "small_pay"
datatest = datatest.drop(['small_pay1', 'small_pay2', 'small_pay3', 'small_pay4', 'small_pay5', 'small_pay6'], axis=1)

# Reemplazar valores de pay1 a pay6 con nuevos valores
for i in range(1, 7):
    datatest['pay'+str(i)] = datatest['pay'+str(i)].replace({-1:0, 1:1, 2:1, 3:1, 4:1, 5:1, 6:1, 7:1, 8:1})

# Suponiendo que ya tienes las etiquetas de entrenamiento (y_train)
y_train = pd.read_csv('ruta/y_train.csv')  # Este es un ejemplo, reemplaza con la ruta correcta

# Crear el pipeline con el escalador y el modelo
pipeline = Pipeline(steps=[
    ('scaler', RobustScaler()),            # Escalado con RobustScaler
    ('model', LogisticRegression())        # Modelo de regresión logística
])

# Entrenar el pipeline
pipeline.fit(datatest, y_train)

# Guardar el pipeline en un archivo con pickle
with open('pipeline_logistic_model.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

# ----------- Para usar el pipeline guardado más adelante ------------ #

# Cargar el pipeline desde el archivo
with open('pipeline_logistic_model.pkl', 'rb') as f:
    loaded_pipeline = pickle.load(f)

# Leer los datos de test
X_test = pd.read_csv('ruta/datatest.csv')  # Lee los datos de prueba
# Hacer predicciones con el pipeline cargado
y_pred = loaded_pipeline.predict(X_test)


In [192]:
# Preprocessing and modeling pipeline to pickle file

class PreprocessingPipeline(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X.columns = ['id', 'limit_bal', 'sex', 'education', 'marriage', 'age', 'pay1', 'pay2', 'pay3', 'pay4', 'pay5', 'pay6', 
                     'bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6', 
                     'pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']
        
        X['education'] = X['education'].replace({0:5, 6:5})
        X['marriage'] = X['marriage'].replace({0:3})

        for i in range(1, 7):
            X['pay'+str(i)] = X['pay'+str(i)].replace({-2:-1, 0:-1})

        for i in range(1, 7):
            X['delay'+str(i)] = X['pay'+str(i)].apply(lambda x: 1 if x > 0 else 0)

        X['n_delays'] = X[['delay1', 'delay2', 'delay3', 'delay4', 'delay5', 'delay6']].sum(axis=1)

        X = X.drop(['delay1', 'delay2', 'delay3', 'delay4', 'delay5', 'delay6'], axis=1)

        X['longest_delay'] = X[['pay1', 'pay2', 'pay3', 'pay4', 'pay5', 'pay6']].max(axis=1)

        X['avg_bill'] = X[['bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6']].mean(axis=1)

        X['highest_bill'] = X[['bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6']].max(axis=1)

        X['debt/limit'] = X['highest_bill'] / X['limit_bal']

        X['avg_pay'] = X[['pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']].mean(axis=1)

        X['avg pay/debt'] = np.where(X['avg_bill'] == 0, 0, X['avg_pay'] / X['avg_bill'])

        for i in range(1, 7):
            X['pay'+str(i)] = X['pay'+str(i)].replace({-1:0, 1:1, 2:1, 3:1, 4:1, 5:1, 6:1, 7:1, 8:1})

        X['longest_delay'] = X['longest_delay'].apply(lambda x: 0 if x == -1 else x)

        for i in range(1, 7):
            X['small_pay'+str(i)] = np.where(X['pay_amt'+str(i)] < 0.05*X['bill_amt'+str(i)], 1, 0)

        X['n_small_pays'] = X[['small_pay1', 'small_pay2', 'small_pay3', 'small_pay4', 'small_pay5', 'small_pay6']].sum(axis=1)

        X = X.drop(['small_pay1', 'small_pay2', 'small_pay3', 'small_pay4', 'small_pay5', 'small_pay6'], axis=1)

        return X.drop(['id'], axis=1)
    
class ModelPipeline(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.pipeline = Pipeline(steps=[
            ('scaler', RobustScaler()),            # Escalado con RobustScaler
            ('model', LogisticRegression(penalty='l1', solver='liblinear'))        # Modelo de regresión logística
        ])

    def fit(self, X, y=None):
        self.pipeline.fit(X, y)
        return self

    def predict(self, X, y=None):
        return self.pipeline.predict(X)

    def predict_proba(self, X, y=None):
        return self.pipeline.predict_proba(X)

    def save(self, path):
        with open(path, 'wb') as f:
            pickle.dump(self.pipeline, f)

    def load(self, path):
        with open(path, 'rb') as f:
            self.pipeline = pickle.load(f)
        return self
    
# Crear el pipeline

pipeline = Pipeline(steps=[
    ('preprocessing', PreprocessingPipeline()),
    ('model', ModelPipeline())
])

# Entrenar el pipeline

X_train = pd.read_csv('./train_p3.csv')
y_train = X_train['default payment next month']
X_train = X_train.drop(['default payment next month'], axis=1)

#train test split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

pipeline.fit(X_train, y_train)

# Guardar el pipeline en un archivo con pickle

with open('HOLAA.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

In [7]:
# Definir las clases PreprocessingPipeline y ModelPipeline


class PreprocessingPipeline(BaseEstimator, TransformerMixin):
    def __init__(self): 
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X.columns = ['id', 'limit_bal', 'sex', 'education', 'marriage', 'age', 'pay1', 'pay2', 'pay3', 'pay4', 'pay5', 'pay6', 
                     'bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6', 
                     'pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']
        
        X['education'] = X['education'].replace({0:5, 6:5})
        X['marriage'] = X['marriage'].replace({0:3})

        for i in range(1, 7):
            X['pay'+str(i)] = X['pay'+str(i)].replace({-2:-1, 0:-1})

        for i in range(1, 7):
            X['delay'+str(i)] = X['pay'+str(i)].apply(lambda x: 1 if x > 0 else 0)

        X['n_delays'] = X[['delay1', 'delay2', 'delay3', 'delay4', 'delay5', 'delay6']].sum(axis=1)

        X = X.drop(['delay1', 'delay2', 'delay3', 'delay4', 'delay5', 'delay6'], axis=1)

        X['longest_delay'] = X[['pay1', 'pay2', 'pay3', 'pay4', 'pay5', 'pay6']].max(axis=1)

        X['avg_bill'] = X[['bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6']].mean(axis=1)

        X['highest_bill'] = X[['bill_amt1', 'bill_amt2', 'bill_amt3', 'bill_amt4', 'bill_amt5', 'bill_amt6']].max(axis=1)

        X['debt/limit'] = X['highest_bill'] / X['limit_bal']

        X['avg_pay'] = X[['pay_amt1', 'pay_amt2', 'pay_amt3', 'pay_amt4', 'pay_amt5', 'pay_amt6']].mean(axis=1)

        X['avg pay/debt'] = np.where(X['avg_bill'] == 0, 0, X['avg_pay'] / X['avg_bill'])

        for i in range(1, 7):
            X['pay'+str(i)] = X['pay'+str(i)].replace({-1:0, 1:1, 2:1, 3:1, 4:1, 5:1, 6:1, 7:1, 8:1})

        X['longest_delay'] = X['longest_delay'].apply(lambda x: 0 if x == -1 else x)

        for i in range(1, 7):
            X['small_pay'+str(i)] = np.where(X['pay_amt'+str(i)] < 0.05*X['bill_amt'+str(i)], 1, 0)

        X['n_small_pays'] = X[['small_pay1', 'small_pay2', 'small_pay3', 'small_pay4', 'small_pay5', 'small_pay6']].sum(axis=1)

        X = X.drop(['small_pay1', 'small_pay2', 'small_pay3', 'small_pay4', 'small_pay5', 'small_pay6'], axis=1)

        return X.drop(['id'], axis=1)

In [8]:
#Pipeline

pipeline = Pipeline(steps=[
('preprocessing', PreprocessingPipeline()),
('scaler', RobustScaler()),
('model', LogisticRegression(penalty='l1', solver='liblinear'))])

In [10]:
#Entrenar el pipeline

X = pd.read_csv('./train_p3.csv')
y = X['default payment next month']
X = X.drop(['default payment next month'], axis=1)

In [11]:
#train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing', PreprocessingPipeline()),
                ('scaler', RobustScaler()),
                ('model',
                 LogisticRegression(penalty='l1', solver='liblinear'))])

In [12]:
# Guardar el pipeline en un archivo con pickle

with open('JMRF.pkl', 'wb') as f:
    pickle.dump(pipeline, f)